In [ ]:
!pip install adal
!pip install Office365-REST-Python-Client
!python -m pip install elasticsearch
!python -m pip install eland

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 2.6MB 8.3MB/s 
     |████████████████████████████████| 296kB 5.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 327kB 5.8MB/s 
     |████████████████████████████████| 204kB 6.0MB/s 


In [ ]:
import pandas as pd
import eland as ed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tempfile
import json
from six.moves.urllib.parse import urlencode, quote
from datetime import datetime
from urllib.request import urlopen
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.runtime.auth.user_credential import UserCredential
from office365.runtime.http.request_options import RequestOptions

from elasticsearch import Elasticsearch
# Import standard test settings for consistent results
from eland.conftest import *

# Loading code <utils.py> from github repository.
url_code_github = "https://raw.githubusercontent.com/hoat23/ElasticSearch/master/bin/utils.py"
code_str = urlopen(url_code_github).read()
code_str = code_str.decode('utf-8')
exec(  code_str  )

In [1]:
#CAMBIAR LOS PARAMETROS Y VARIABLES
SHAREPOINT_USER = "deiner.zapata@microsoft.pe"
SHAREPOINT_PASS = "mysecretpasword"
SHAREPOINT_URL = "https://deiner.sharepoint.com"
SITE_URL = "{0}/sites/GrupoDeiner".format(SHAREPOINT_URL)
                 
PATH_DIRECTORY = "sites/GrupoDeiner/Documentos compartidos/Elastic/Reportes por grupo"

LIST_FILES = [
#  RELATIVE_PATH                                          EXCEL_FILENAME                   EXCEL_SHEETNAME
  [""                                                 , "Sector 01.xlsx"               , "Hoja2"          ],
  [""                                                 , "Notas.xlsx"                   , ""               ],
  ["Reportes Consolidados/Procesados/00 Grupo01"      , "Grupo01.xlsx"                 , ""               ],
  ["Reportes Consolidados/Procesados/01 Grupo02"      , "Grupo02.xlsx"                 , ""               ],
  ["Reportes Consolidados/Procesados/02 Grupo03"      , "Grupo03.xlsx"                 , ""               ]
]
# NOTA: Fullpath is [ PATH_DIRECTORY + RELATIVE_PATH ]

In [ ]:
def connect_to_sharepoint(url, user, pasw):
  sharepoint_conector = ClientContext(url).with_credentials(UserCredential(user, pasw))
  web = sharepoint_conector.web
  sharepoint_conector.load(web)
  sharepoint_conector.execute_query()
  print("connect_to_sharepoint | web title: {0}".format(web.properties['Title']))
  return sharepoint_conector

def download_file(url_path_relative,url_conector):
  download_path = os.path.join(tempfile.mkdtemp(), os.path.basename(url_path_relative))
  with open(download_path, "wb") as local_file:
      file = url_conector.web.get_file_by_server_relative_url(url_path_relative).download(local_file).execute_query()
  print("[Ok] Archivo cargado: {0}".format(download_path))
  return download_path

def download_list_files(list_files, path_directory, url_conector):
  list_download_paths = []
  for one_file in list_files:
    if len(one_file[0])==0:
      url_path_relative =  quote( "/{0}/{1}".format(path_directory,one_file[1]) )
    else:
      url_path_relative =  quote( "/{0}/{1}/{2}".format(path_directory,one_file[0],one_file[1]) )
    list_download_paths.append( [ download_file(url_path_relative, url_conector) , one_file[2] ] )
  return list_download_paths

In [ ]:
sharepoint_conector = connect_to_sharepoint(SITE_URL,SHAREPOINT_USER, SHAREPOINT_PASS)
list_download_paths = download_list_files(LIST_FILES, PATH_DIRECTORY, sharepoint_conector)

In [ ]:
def listar_sheetnames_in_excel(list_files_paths):
  for one_file_path in list_files_paths:
    xls = pd.ExcelFile(one_file_path[0], engine='xlrd')
    print("{0:60s} | sheetnames = {1}".format(one_file_path[0], xls.sheet_names))

def listar_columns_in_excel(list_files_paths):
  for one_file_path in list_files_paths:
    xls = pd.ExcelFile(one_file_path[0], engine='xlrd')
    df = pd.read_excel(xls, xls.sheet_names[0])
    print("{0}".format( list(df.columns)) )

print("\nSHEETNAMES")
listar_sheetnames_in_excel(list_download_paths)
#print("\nCOLUMNAS")
#listar_columns_in_excel(list_download_paths)

In [ ]:
CLUSTER_ENDPOINT = "https://123123.us-central1.gcp.cloud.es.io:9243"
ID = "123123123123"
API_KEY = "adfadfasdfasdfdadf"

In [ ]:
# Connect to an Elastic Cloud instance
es = Elasticsearch(
    [CLUSTER_ENDPOINT],
    api_key=( ID , API_KEY),
)

print_json(es.info())

In [ ]:
import re

def excel_to_elasticsearch(excel_filename, excel_sheetname, index_name, es_conector):
  xls = pd.ExcelFile(excel_filename, engine='xlrd')
  if len(excel_sheetname)==0:
    # By defaul load the first sheet in excel
    excel_sheetname = xls.sheet_names[0]
  print("\nexcel_to_elasticsearch | filename [{0}] | sheetname [{1}]".format(excel_filename, excel_sheetname))
  df_pandas = pd.read_excel(xls, excel_sheetname)
  df_pandas = df_pandas.replace({np.NAN: None, np.inf: None})
  df_pandas = df_pandas.astype(object).where(pd.notnull(df_pandas),None)
  df_pandas.info()
  index_name = excel_sheetname.replace("ñ","n")
  index_name =  re.sub( '[^a-zA-Z0-9\n\.]', '', index_name )
  index_name = index_name.lower()
  print("\nexcel_to_elasticsearch | index_name [{0}]".format(index_name))
  df_eland = ed.pandas_to_eland(
    pd_df= df_pandas,
    es_client= es_conector,
    es_dest_index= index_name,
    es_if_exists= "replace",# If the index already exists what should we do?
    es_refresh= True,# Wait for data to be indexed before returning
  )
  df_eland.info()
  del df_pandas, xls, df_eland

def load_list_paths_to_elasticsearch(list_files_paths, es_conector):
  for one_file_path in list_files_paths:
    excel_filename = one_file_path[0]
    excel_sheetname = one_file_path[1]
    excel_to_elasticsearch(excel_filename, excel_sheetname, "", es_conector)

load_list_paths_to_elasticsearch(list_download_paths, es)